![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/openvino/HuggingFace_OpenVINO_in_Spark_NLP_CamemBertForTokenClassification.ipynb)

# Import OpenVINO CamemBertForTokenClassification  models from HuggingFace 🤗 into Spark NLP 🚀

This notebook provides a detailed walkthrough on optimizing and exporting CamemBertForTokenClassification  models from HuggingFace for use in Spark NLP, leveraging the various tools provided in the [Intel OpenVINO toolkit](https://www.intel.com/content/www/us/en/developer/tools/openvino-toolkit/overview.html) ecosystem.

Let's keep in mind a few things before we start 😊

- OpenVINO support was introduced in  `Spark NLP 5.4.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for CamemBertForTokenClassification from CamemBertForTokenClassification and they have to be in `Token Classification` category.

## 1. Export and Save the HuggingFace model

- Let's install `transformers` and `openvino` packages with other dependencies. You don't need `openvino` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install -q --upgrade transformers==4.48.3 optimum[openvino]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.6/342.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00


[Optimum Intel](https://github.com/huggingface/optimum-intel?tab=readme-ov-file#openvino) is the interface between the Transformers library and the various model optimization and acceleration tools provided by Intel. HuggingFace models loaded with optimum-intel are automatically optimized for OpenVINO, while being compatible with the Transformers API.
- To load a HuggingFace model directly for inference/export, just replace the `AutoModelForXxx` class with the corresponding `OVModelForXxx` class. We can use this to import and export OpenVINO models with `from_pretrained` and `save_pretrained`.
- By setting `export=True`, the source model is converted to OpenVINO IR format on the fly.
- We'll use [Jean-Baptiste/camembert-ner](https://huggingface.co/Jean-Baptiste/camembert-ner) model from HuggingFace as an example and load it as a `OVModelForTokenClassification`, representing an OpenVINO model.
- In addition to the OVModelForTokenClassification model, we also need to save the `AutoTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [ ]:
from transformers import CamembertTokenizer
from optimum.intel import OVModelForTokenClassification

MODEL_NAME = 'Jean-Baptiste/camembert-ner'
EXPORT_PATH = f"ov_models/{MODEL_NAME}"

ov_model = OVModelForTokenClassification.from_pretrained(MODEL_NAME, export=True)
ov_model.save_pretrained(EXPORT_PATH)

tokenizer = CamembertTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(EXPORT_PATH)

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/892 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/269 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


('onnx_models/Jean-Baptiste/camembert-ner/tokenizer_config.json',
 'onnx_models/Jean-Baptiste/camembert-ner/special_tokens_map.json',
 'onnx_models/Jean-Baptiste/camembert-ner/sentencepiece.bpe.model',
 'onnx_models/Jean-Baptiste/camembert-ner/added_tokens.json')

Let's have a look inside these two directories and see what we are dealing with:

In [6]:
!ls -l {EXPORT_PATH}

total 430220
-rw-r--r-- 1 root root        22 Jun 21 02:52 added_tokens.json
drwxr-xr-x 2 root root      4096 Jun 21 02:52 assets
-rw-r--r-- 1 root root       936 Jun 21 02:52 config.json
-rw-r--r-- 1 root root 440145092 Jun 21 02:52 openvino_model.bin
-rw-r--r-- 1 root root    374491 Jun 21 02:52 openvino_model.xml
-rw-r--r-- 1 root root       354 Jun 21 02:52 special_tokens_map.json
-rw-r--r-- 1 root root      1794 Jun 21 02:52 tokenizer_config.json


- We need to move `sentencepiece.bpe.model` from the tokenizer to assets folder which Spark NLP will look for
- In addition to vocabs, we also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [5]:
!mkdir -p {EXPORT_PATH}/assets

labels = ov_model.config.id2label
labels = [value for key,value in sorted(labels.items(), reverse=False)]

with open(EXPORT_PATH + '/assets/labels.txt', 'w') as f:
    f.write('\n'.join(labels))

!mv {EXPORT_PATH}/sentencepiece.bpe.model {EXPORT_PATH}/assets/

In [7]:
!ls -lR {EXPORT_PATH}

onnx_models/Jean-Baptiste/camembert-ner:
total 430220
-rw-r--r-- 1 root root        22 Jun 21 02:52 added_tokens.json
drwxr-xr-x 2 root root      4096 Jun 21 02:52 assets
-rw-r--r-- 1 root root       936 Jun 21 02:52 config.json
-rw-r--r-- 1 root root 440145092 Jun 21 02:52 openvino_model.bin
-rw-r--r-- 1 root root    374491 Jun 21 02:52 openvino_model.xml
-rw-r--r-- 1 root root       354 Jun 21 02:52 special_tokens_map.json
-rw-r--r-- 1 root root      1794 Jun 21 02:52 tokenizer_config.json

onnx_models/Jean-Baptiste/camembert-ner/assets:
total 796
-rw-r--r-- 1 root root     26 Jun 21 02:52 labels.txt
-rw-r--r-- 1 root root 810912 Jun 21 02:52 sentencepiece.bpe.model


- Voila! We have our `sentencepiece.bpe.model` and `labels.txt` inside assets directory

## Import and Save CamemBertForTokenClassification in Spark NLP


- Let's install and setup Spark NLP in Google Colab.
- For this example, we'll use specific versions of `pyspark` and `spark-nlp` that we've already tested with this transformer model to make sure everything runs smoothly.

In [8]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 5.5 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [9]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: {}".format(sparknlp.version()))
print("Apache Spark version: {}".format(spark.version))

Spark NLP version: 5.5.3
Apache Spark version: 3.5.4


- Let's use `loadSavedModel` functon in `CamemBertForTokenClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `CamemBertForTokenClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [10]:
from sparknlp.annotator import CamemBertForTokenClassification

tokenClassifier = CamemBertForTokenClassification.loadSavedModel(
     f"{EXPORT_PATH}",
     spark
 )\
  .setInputCols(["document",'token'])\
  .setOutputCol("ner")\
  .setCaseSensitive(True)\
  .setMaxSentenceLength(128)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [11]:
tokenClassifier.write().overwrite().save("./{}_spark_nlp_openvino".format(EXPORT_PATH))

Let's clean up stuff we don't need anymore

In [12]:
!rm -rf {EXPORT_PATH}

Awesome 😎  !

This is your CamemBertForTokenClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [13]:
! ls -l {EXPORT_PATH}_spark_nlp_openvino

total 431068
-rw-r--r-- 1 root root 440587086 Jun 21 02:57 camembert_classification_openvino
-rw-r--r-- 1 root root    810912 Jun 21 02:57 camembert_spp
drwxr-xr-x 3 root root      4096 Jun 21 02:57 fields
drwxr-xr-x 2 root root      4096 Jun 21 02:57 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny CamemBertForTokenClassification model 😊

In [14]:
tokenClassifier_loaded = CamemBertForTokenClassification.load("./{}_spark_nlp_openvino".format(EXPORT_PATH))\
  .setInputCols(["document",'token'])\
  .setOutputCol("ner")

You can see what labels were used to train this model via `getClasses` function:

In [15]:
tokenClassifier_loaded.getClasses()

['I-ORG', 'I-MISC', 'I-LOC', 'I-PER', 'O']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [16]:
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer, NerConverter
from pyspark.ml import Pipeline

document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

converter = NerConverter() \
    .setInputCols(["document", "token", "ner"]) \
    .setOutputCol("ner_chunk")

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    tokenClassifier_loaded,
    converter
])

example = spark.createDataFrame([
    ["""Emmanuel Macron a rencontré Angela Merkel à Berlin pour parler de l'Union Européenne.
    Le sommet s'est tenu au Bundestag avec la Commission Européenne.
    Airbus et Renault ont été mentionnés pour leur rôle économique.
    Kylian Mbappé a brillé en Ligue 1 avec le PSG.
    Zinédine Zidane s'est confié à L'Équipe.
    La NASA et SpaceX préparent une mission sur Mars avec le soutien d’Elon Musk.
    Le CNES, basé à Toulouse, y participe.
    Jean-Pierre Jeunet adapte Victor Hugo au cinéma.
    Christine and the Queens chantera bientôt au Zénith de Paris."""]
], ["text"])

model = pipeline.fit(example)
result = model.transform(example)

result.selectExpr("explode(ner_chunk) as chunk").selectExpr(
    "chunk.result as text",
    "chunk.metadata['entity'] as entity"
).show(truncate=False)

+---------------------+------+
|text                 |entity|
+---------------------+------+
|Emmanuel Macron      |PER   |
|Angela Merkel        |PER   |
|Berlin               |LOC   |
|Européenne           |ORG   |
|Bundestag            |ORG   |
|Commission Européenne|ORG   |
|Airbus               |ORG   |
|Renault              |ORG   |
|Kylian Mbappé        |PER   |
|Ligue 1              |ORG   |
|PSG                  |ORG   |
|Zinédine Zidane      |PER   |
|L'Équipe             |ORG   |
|NASA                 |ORG   |
|SpaceX               |ORG   |
|Mars                 |LOC   |
|Musk                 |PER   |
|CNES                 |ORG   |
|Toulouse             |LOC   |
|Jean-Pierre Jeunet   |PER   |
+---------------------+------+
only showing top 20 rows



That's it! You can now go wild and use hundreds of `CamemBertForTokenClassification` models from HuggingFace 🤗 in Spark NLP 🚀
